# Demographic Structure and Amenity Merge

This notebook adds the demographic context needed to interpret the amenity supply data. It loads Community Area level information on dependents, working age share, per capita income, and hardship index. Community Area names are standardized to match the format used in the amenity file so the merge is consistent.

The notebook computes working age share and then merges the demographic fields with the amenity variables produced in the data pipeline. The output is a combined dataset that contains both supply and demand side indicators. This file supports all later analysis, including the saturation index, similarity networks, and spatial patterns in Chicago.


In [1]:
import pandas as pd


In [7]:
# Load supply data
amenities = pd.read_csv("amenities_by_CA.csv")

# Normalize case for amenities
amenities["ca_name"] = amenities["ca_name"].str.upper().str.strip()

# Load ACS demographic file
demographics = pd.read_csv("acs_chicago_by_CA.csv")




In [8]:

# Normalize case for demographics
demographics["COMMUNITY AREA NAME"] = demographics["COMMUNITY AREA NAME"].str.upper().str.strip()

# Rename fields
demographics = demographics.rename(columns={
    "Community Area Number": "ca_num",
    "COMMUNITY AREA NAME": "ca_name",
    "PERCENT AGED UNDER 18 OR OVER 64": "pct_dependents",
    "PER CAPITA INCOME ": "per_capita_income",
    "HARDSHIP INDEX": "hardship_index"
})

# Compute working age share
demographics["pct_working_age"] = 100 - demographics["pct_dependents"]

# Select columns for merge
demo_cols = [
    "ca_num",
    "ca_name",
    "pct_dependents",
    "pct_working_age",
    "per_capita_income",
    "hardship_index"
]

# Merge
merged = amenities.merge(
    demographics[demo_cols],
    on=["ca_num", "ca_name"],
    how="left"
)

# Save output
merged.to_csv("demographics_amenities.csv", index=False)

merged.head()

,ca_num,ca_name,business_license_count,food_inspections_count,liquor_license_count,building_permits_count,pct_dependents,pct_working_age,per_capita_income,hardship_index
0,1,ROGERS PARK,2.0,5.0,17.0,9.0,27.5,72.5,23939.0,39.0
1,2,WEST RIDGE,16.0,11.0,12.0,6.0,38.5,61.5,23040.0,46.0
2,3,UPTOWN,6.0,9.0,15.0,14.0,22.2,77.8,35787.0,20.0
3,4,LINCOLN SQUARE,11.0,17.0,15.0,13.0,25.5,74.5,37524.0,17.0
4,5,NORTH CENTER,4.0,7.0,22.0,18.0,26.2,73.8,57123.0,6.0
